In [ ]:
# default_exp stats

In [ ]:
%load_ext autoreload
%autoreload 2

# stats

> A way to access metadata on all the files due for processing.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import json
import os
import datetime
from pathlib import Path
from typing import Any

import pandas as pd
import numpy as np
import fitz

from rich import print
from rich.console import Console
from rich.table import Table

In [ ]:
%load_ext rich

In [ ]:
#export
def get_page_count(filepath: "os.PathLike[Any]") -> int:
    """Gets the page count of a PDF file."""
    pdf_obj = fitz.open(filepath)
    return pdf_obj.page_count

In [ ]:
#export
def get_file_metadata(filepath: "os.PathLike[Any]") -> dict:
    """Gets the metadata associated with a PDF file."""
    pdf_obj = fitz.open(filepath)
    return pdf_obj.metadata

In [ ]:
#export
def add_comma_separation(input: int) -> str:
    """Adds comma-separation for thousands to an integer."""
    return f'{input:,.0f}'

In [ ]:
#export
def has_ocr_layer(filepath: "os.PathLike[Any]") -> bool:
    """Checks whether a particular file has an OCR layer."""
    # TODO: fix this function
#     pdf_obj = fitz.open(filepath)
#     return len(pdf_obj[0].get_text("text")) == 0
    return False

In [ ]:
#export
def get_stats(source_path: "os.PathLike[Any]") -> list:
    """Gathers statistics on the PDF data contained in a particular directory."""
    stats_data = []
    
    files = list(source_path.glob("**/*.pdf")) # searches source_path and all subfolders
    
    for file in files:
        file_data = {
            "filename": file.name,
            "pagecount": get_page_count(file),
            "has_ocr_layer": has_ocr_layer(file),
            "pdf_file_size_bytes": os.path.getsize(file),
            "date_created": datetime.datetime.fromtimestamp(os.path.getctime(file)),
            "date_last_modified": datetime.datetime.fromtimestamp(os.path.getmtime(file)),
            "author": get_file_metadata(file)['author'],
        }
        stats_data.append(file_data)

    return stats_data

In [ ]:
#export
def convert_timestamp(item_date_object):
    """Helper function to convert a datetime object to timestamp when
    needed for a JSON object."""
    if isinstance(item_date_object, (datetime.date, datetime.datetime)):
        return item_date_object.timestamp()

In [ ]:
#export
def get_json_stats(source_path: "os.PathLike[Any]") -> str:
    """Gathers statistics on the PDF data in a directory in JSON format."""
    return json.dumps(get_stats(source_path), default=convert_timestamp)

In [ ]:
#export
def get_dataframe_stats(source_path: "os.PathLike[Any]") -> pd.core.frame.DataFrame:
    """Gathers statistics on the PDF data in a directory as a dataframe."""
    return pd.DataFrame.from_dict(get_stats(source_path))

In [ ]:
#export
def export_stats_as_csv(source_path: "os.PathLike[Any]", destination_path: "os.PathLike[Any]" = Path("./stats.csv")) -> None:
    """Exports statistics on the PDF data as a CSV file."""
    get_dataframe_stats(source_path).to_csv(destination_path)

In this next section I manage to get some makeshift drift detection working with `evidently`:

In [ ]:
from evidently.dashboard import Dashboard
from evidently.tabs import DataDriftTab
from evidently.pipeline.column_mapping import ColumnMapping

source1 = Path("/Users/strickvl/Desktop/NL")
source2 = Path("/Users/strickvl/Desktop/machine-learning-flashcards")

data_1 = get_dataframe_stats(source1)
data_2 = get_dataframe_stats(source2)

data_types_dict = {
    'filename': str,
    "pagecount": np.number,
    'has_ocr_layer': np.number,
    'pdf_file_size_bytes': np.number,
    'author': str,
}

data_1['date_created'] = pd.to_datetime(data_1['date_created'])
data_1['date_last_modified'] = pd.to_datetime(data_1['date_last_modified'])
data_1 = data_1.astype(data_types_dict)

data_2['date_created'] = pd.to_datetime(data_2['date_created'])
data_2['date_last_modified'] = pd.to_datetime(data_2['date_last_modified'])
data_2 = data_2.astype(data_types_dict)

cols_to_drop = ['date_created', 'date_last_modified', "filename", "author"]
data_1.drop(cols_to_drop, axis=1, inplace=True)
data_2.drop(cols_to_drop, axis=1, inplace=True)

# export_stats_as_csv(source1, Path("./tryout/stats1.csv"))
# export_stats_as_csv(source2, Path("./tryout/stats2.csv"))

# df1 = pd.read_csv("./tryout/stats1.csv")
# df2 = pd.read_csv("./tryout/stats1.csv")

data_drift_report = Dashboard(tabs=[DataDriftTab()])
data_drift_report.calculate(data_1, data_2)
data_drift_report.show()

Loading...

In [ ]:
#export
def display_stats(stats_list: list) -> None:
    """Displays statistics on the PDF data contained in a particular directory."""
    table = Table(title="Stats for your PDF Files")

    table.add_column("PageCount", justify="right", style="green")
    table.add_column("Filename", justify="left", style="cyan", no_wrap=True, max_width=50)
    table.add_column("ocr_layer", justify="left", style="blue")
    table.add_column("pdf_file_size_bytes", justify="left", style="purple")
    table.add_column("author", justify="left", style="black")
    page_count = 0
    file_count = 0

    for item in stats_list:
        page_count += item['pagecount']
        file_count += 1
        
        if file_count <= 45:
            table.add_row(
                str(item['pagecount']),
                item['filename'],
                str(item['has_ocr_layer']),
                str(item['pdf_file_size_bytes']),
                item['author'],
            )
    if file_count > 45:
        table.add_row("...", "...", "...", "...", "...")
        
    console = Console()
    console.print(table)
    
    bold_str_count = f"[bold red]{add_comma_separation(page_count)}"
    console.print("[bold red]TOTAL PAGECOUNT:", bold_str_count)

In [ ]:
source = Path("/Users/strickvl/Desktop/machine-learning-flashcards")

display_stats(get_stats(source))

                                  Stats for your PDF Files                                   
┏━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓
┃ PageCo… ┃ Filename                                           ┃ ocr_la… ┃ pdf_fi… ┃ author ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩
│       1 │ Residual_Sum_Of_Squares.pdf                        │ False   │ 1045078 │        │
│       1 │ Basic_Parts_Of_Deep_Learning.pdf                   │ False   │ 871672  │        │
│       1 │ Conditioning.pdf                                   │ False   │ 368812  │        │
│       1 │ DBSCAN.pdf                                         │ False   │ 779406  │        │
│       1 │ Youdens_J_Statistic.pdf                            │ False   │ 1181340 │        │
│       1 │ Handling_Imbalanced_Classes_In_Support_Vector_Mac… │ False   │ 423704  │        │
│       1 │ Word2Vec.pdf                                       │ False   │ 518507  │        │
│       1 │ Bayes_Theorem.pdf                                  │ False   │ 448065  │        │
│       1 │ Uniform_Distribution.pdf                           │ False   │ 574346  │        │
│       1 │ Matthews_Correlation_Coefficient.pdf               │ False   │ 501665  │        │
│       1 │ Bias-Variance_Tradeoff.pdf                         │ False   │ 1172740 │        │
│       1 │ Hidden_Layer.pdf                                   │ False   │ 1335063 │        │
│       1 │ K-Nearest_Neighbors_Tips_And_Tricks.pdf            │ False   │ 558009  │        │
│       1 │ How_To_Choose_Hidden_Unit_Activation_Functions.pdf │ False   │ 475676  │        │
│       1 │ Bayes_Error.pdf                                    │ False   │ 1518133 │        │
│       1 │ Softmax_Normalization.pdf                          │ False   │ 1086385 │        │
│       1 │ Anscombes_Quartet.pdf                              │ False   │ 1595618 │        │
│       1 │ Saddle_Point.pdf                                   │ False   │ 708805  │        │
│       1 │ k-Nearest_Neighbors.pdf                            │ False   │ 1245264 │        │
│       1 │ Early_Stopping_Advantages.pdf                      │ False   │ 1053443 │        │
│       1 │ Fowlkes-Mallows.pdf                                │ False   │ 1622939 │        │
│       1 │ Gradient.pdf                                       │ False   │ 1849334 │        │
│       1 │ Capacity.pdf                                       │ False   │ 577765  │        │
│       1 │ Regularization.pdf                                 │ False   │ 1018306 │        │
│       1 │ Supervised_Deep_Learning_Rule_Of_Thumb.pdf         │ False   │ 1299819 │        │
│       1 │ Decision_Trees.pdf                                 │ False   │ 1203288 │        │
│       1 │ Confusion_Matrix.pdf                               │ False   │ 1442159 │        │
│       1 │ Minimum_Of_A_Loss_Function.pdf                     │ False   │ 660131  │        │
│       1 │ Variance.pdf                                       │ False   │ 665448  │        │
│       1 │ AIC.pdf                                            │ False   │ 900340  │        │
│       1 │ Hinge_Loss.pdf                                     │ False   │ 1052360 │        │
│       1 │ Stop_Words.pdf                                     │ False   │ 861381  │        │
│       1 │ Sensitivity.pdf                                    │ False   │ 957821  │        │
│       1 │ No_Free_Lunch_Theorem.pdf                          │ False   │ 1065637 │        │
│       1 │ Bias.pdf                                           │ False   │ 615440  │        │
│       1 │ Overfit_Vs_Underfit.pdf                            │ False   │ 868425  │        │
│       1 │ Probability_Density_Function.pdf                   │ False   │ 1142624 │        │
│       1 │ Noisy_ReLU.pdf                                     │ False   │ 648507  │        │
│       1 │ Categorical_Feature.pdf                 

TOTAL PAGECOUNT: 900